In [80]:
# For Installation (Required)
!pip install pyLDAvis
!pip install glove-python-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [81]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import re
import string
import nltk
import gensim
import time
import pickle
import warnings
warnings.filterwarnings('ignore')

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

from gensim import corpora,models
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models import CoherenceModel
from gensim.models import Word2Vec,FastText
from gensim.test.utils import get_tmpfile

from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

from glove import Glove
from glove import Corpus

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA

from scipy.spatial import distance

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [82]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [83]:
#Import Whole Dataset
mobile_data=pd.read_csv('/content/drive/Shareddrives/DSCI-644-Team-5/PrimaryDataset/mobile_preproccessed_whole_dataset.csv')

#Cleaning the Data
mobile_data = mobile_data.reset_index()
mobile_data = mobile_data.drop(columns = ["index","Unnamed: 0"])
mobile_data = mobile_data.drop_duplicates()
mobile_data = mobile_data.dropna(axis=0, subset=['Bug ID'])

#To show Data
mobile_data.head()

,Bug ID,Type,Summary,Product,Component,Status,Resolution,Updated,Bug Id,Description,Duplicate_Bug_Ids
0,1668376,defect,Test failure in dom/base/test/test_window_clos...,GeckoView,General,RESOLVED,FIXED,2020-10-14 03:01:38,1668376,the symptom was that the broadcastchannel me...,NaN
1,1631754,defect,PanZoomController erroneously returns INPUT_RE...,GeckoView,General,RESOLVED,FIXED,2020-05-23 04:47:46,1631754,for websites which have their own event handle...,NaN
2,1677838,defect,NumberFormatExceptionjava.lang.Integer in pars...,GeckoView,General,RESOLVED,FIXED,2021-01-06 13:28:48,1677838,we are seeing these crash being reported on se...,NaN
3,1686100,enhancement,Route Service.onTrimMemory notifications throu...,GeckoView,General,RESOLVED,FIXED,2021-01-12 02:17:23,1686100,since android may send these to the services h...,NaN
4,1630229,defect,Crash in [@ java.lang.AssertionError: at org.m...,GeckoView,General,RESOLVED,FIXED,2020-05-28 05:17:09,1630229,this bug is for crash report bp ebbf ...,NaN


In [84]:
#To show Info
mobile_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5320 entries, 0 to 5319
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Bug ID             5320 non-null   int64  
 1   Type               5320 non-null   object 
 2   Summary            5320 non-null   object 
 3   Product            5320 non-null   object 
 4   Component          5320 non-null   object 
 5   Status             5320 non-null   object 
 6   Resolution         5320 non-null   object 
 7   Updated            5320 non-null   object 
 8   Bug Id             5320 non-null   int64  
 9   Description        5320 non-null   object 
 10  Duplicate_Bug_Ids  562 non-null    float64
dtypes: float64(1), int64(2), object(8)
memory usage: 498.8+ KB


# **On complete JDT data**

In [85]:
#Text Cleaning to Remove Punctuations
def clean_text_round_1(text):
    text = re.sub(r'\w*\d\w*', '', text)
    text = re.sub(r'\w*\f\w*', '', text)
    text = re.sub(r'\(.*?\)', '', text)
    text = re.sub(r'\[.*]\)', '', text)
    text = text.lower()
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    return text

round1 = lambda x: clean_text_round_1(x)

#Text Cleaning to Remove Additional Punctuations
def clean_text_round_2(text):
    text = re.sub(r'[‘’“”…]', '', text)
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\t', '', text)
    return text

round2 = lambda x: clean_text_round_2(x)

In [86]:
#To Clean Data in 'Description' Column
mobile_data["Description"]= mobile_data["Description"].str.replace("fixed in HEAD", "", case = False)
mobile_data["Description"]= mobile_data["Description"].str.replace("has been marked as readonly", " ", case = False)

mobile_data = mobile_data.dropna(axis=0, subset=['Description'])

mobile_data['Description'] = mobile_data['Description'].apply(clean_text_round_1)
mobile_data['Description'] = mobile_data['Description'].apply(clean_text_round_2)

#To show Info
mobile_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5320 entries, 0 to 5319
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Bug ID             5320 non-null   int64  
 1   Type               5320 non-null   object 
 2   Summary            5320 non-null   object 
 3   Product            5320 non-null   object 
 4   Component          5320 non-null   object 
 5   Status             5320 non-null   object 
 6   Resolution         5320 non-null   object 
 7   Updated            5320 non-null   object 
 8   Bug Id             5320 non-null   int64  
 9   Description        5320 non-null   object 
 10  Duplicate_Bug_Ids  562 non-null    float64
dtypes: float64(1), int64(2), object(8)
memory usage: 498.8+ KB


In [87]:
#Helper Functions for Preprocessing
def lemmatize(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 5:
            result.append(lemmatize(token))
    return result

In [88]:
#To Preprocess Data in 'Description' Column
mobile_data['Description'] = mobile_data['Description'].map(preprocess)

print('Null Duplicate Bug Ids: ',mobile_data['Duplicate_Bug_Ids'].isnull().sum())

#To save Duplicate Reports in a CSV File
duplicate_reports = mobile_data.dropna(axis=0, subset=['Duplicate_Bug_Ids'])
duplicate_reports.reset_index(drop=True)
duplicate_reports.to_csv('mobile_duplicate_reports.csv')

#Seperating all the master reports into a dataframe
master_reports = mobile_data[mobile_data.isnull().any(axis=1)]
master_reports.reset_index(drop=True)

print('NA Values in Master Report: ', master_reports.Description.isna().sum())

#To save Master Reports in a CSV File
master_reports.to_csv('mobile_master_reports.csv')

Null Duplicate Bug Ids:  4758
NA Values in Master Report:  0


In [89]:
#Import Master Reports
master_reports = pd.read_csv('mobile_master_reports.csv')
master_reports = master_reports.drop(columns=['Unnamed: 0'])

#To Preprocess Data in 'Description' Column
master_reports['Description'] = master_reports['Description'].map(preprocess)

In [90]:
#To Create a Dictionary
dictionary = gensim.corpora.Dictionary(master_reports['Description'])
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

#To Create BoW a Dictionary
bow_corpus = [dictionary.doc2bow(doc) for doc in master_reports['Description']]

In [91]:
#To Open Pickle File
file_bow = open('mobile_bow_corpus.pickle', 'wb')

#To dump BoW data in Pickle File
pickle.dump(bow_corpus, file_bow)

#To Open Pickle File
file_dict = open('mobile_dictionary.pickle', 'wb')

#To dump Dictionary data in Pickle File
pickle.dump(dictionary, file_dict)

In [92]:
#Parameters for LDA Model
corpus = bow_corpus
no_of_topics = 10
dictionary = dictionary
p = 20
k = 2
epochs = 100

#Training the LDA model on the BoW corpus
lda_model = gensim.models.LdaMulticore(corpus, num_topics=no_of_topics, id2word=dictionary, passes=p, workers=k, iterations=epochs)

In [93]:
#Saving the Model
lda_model.save('mobile_lda_model.model')

In [94]:
#Evaluation of Model

#Perplexity
print('Perplexity: ', lda_model.log_perplexity(bow_corpus))  

#Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=mobile_data['Description'], dictionary=dictionary, coherence='c_v')
print('\nCoherence Score: ', coherence_model_lda.get_coherence())

Perplexity:  -4.373968116529857

Coherence Score:  0.598303448441005


In [95]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, bow_corpus, dictionary)

In [96]:
#Visualization
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.379316 -0.120939       1        1  49.376022
6     -0.105909 -0.165946       2        1   8.535589
0      0.149609  0.150563       3        1   8.153274
4      0.353330 -0.055031       4        1   7.485146
8     -0.195395 -0.173743       5        1   5.400966
3     -0.201053 -0.103612       6        1   5.337681
7     -0.063543  0.230423       7        1   5.211858
9     -0.036391  0.166236       8        1   3.710973
1     -0.148099 -0.038763       9        1   3.447619
2     -0.131866  0.110811      10        1   3.340872, topic_info=          Term          Freq         Total Category  logprob  loglift
27     mozilla  19142.000000  19142.000000  Default  30.0000  30.0000
37   geckoview  18121.000000  18121.000000  Default  29.0000  29.0000
199    firefox   2029.000000   2029.000000  Default  28.0000  28.0000
30      result   1805.000000   1805.000000  Default  27.0000  27.0000
15     android   4643.000000   4643.000000  Default  26.0000  26.0000
..         ...           ...           ...      ...      ...      ...
199    firefox     88.020193   2029.956379  Topic10  -3.9762   0.2607
282    example     55.325386    323.094394  Topic10  -4.4405   1.6342
49      relate     41.776116    129.180556  Topic10  -4.7214   2.2701
117     enable     45.066050    428.984296  Topic10  -4.6456   1.1456
45      method     47.717705   1138.093043  Topic10  -4.5884   0.2271

[551 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
76        1  0.201166      access
76        2  0.026239      access
76        3  0.293003      access
76        5  0.074344      access
76        6  0.139942      access
...     ...       ...         ...
629       6  0.041450   xcuitests
629       7  0.273568   xcuitests
629      10  0.613456   xcuitests
437       2  0.987881     youtube
55        1  0.998598  zygoteinit

[1238 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 7, 1, 5, 9, 4, 8, 10, 2, 3])

In [97]:
#Creating 10 empty clusters and pushing master reports in each of them based on topic modeling and saving them in individual csv file
for c in range(10):
    exec('topic_{} = pd.DataFrame()'.format(c))
    for i in range(len(master_reports)):
        topic=lda_model[dictionary.doc2bow(master_reports.Description[i])]
        topic= np.asarray(topic)
        if int(topic[np.argmax(topic[:,1]),0])== c:
            exec('topic_{} = topic_{}.append(master_reports.loc[[i]])'.format(c,c))
            exec('topic_{} = topic_{}.reset_index(drop=True)'.format(c,c))
            exec('topic_{}.to_csv("topic_{}.csv")'.format(c,c))

In [98]:
#To Open Pickle File
file_bow = open('mobile_bow_corpus.pickle', 'rb')
bow_corpus = pickle.load(file_bow)

file_dict = open('mobile_dictionary.pickle', 'rb')
dictionary=pickle.load(file_dict)

#To load Trained Model
lda_model =  models.LdaModel.load('mobile_lda_model.model')

In [99]:
#To Create corpus for Word2Vec and FastText models
for i in range(10):
    exec('sent_{} = []'.format(i))
    exec('x= topic_{}'.format(i))
    for j in range(len(x)):
        exec('sent_{}.append(topic_{}.Description[{}])'.format(i,i,j))

for sent in range(10):
    exec('print(len(sent_{}))'.format(sent))

618
345
466
532
137
395
733
673
622
243


In [100]:
#Training GloVe model for each cluster
for cluster in range(10):
    vector_size = 100
    exec('glove_corpus{}=Corpus()'.format(cluster, cluster)) 
    exec('glove_corpus{}.fit(sent_{})'.format(cluster, cluster))
    exec('glove{}= Glove(no_components=vector_size, learning_rate=0.18, alpha=0.75, max_count=100, max_loss=10.0, random_state=None)'.format(cluster, cluster))
    exec('glove{}.fit(glove_corpus{}.matrix, epochs=200, no_threads=3, verbose=True)'.format(cluster, cluster))
    exec('transformer = lambda dictionary2:glove{}.transform_paragraph(words, epochs=1000,ignore_missing=False)'.format(cluster, cluster))
    exec('glove{}.add_dictionary(glove_corpus{}.dictionary)'.format(cluster, cluster))

    #Save the all the models in individual file
    exec('path = get_tmpfile("glove{}.model")'.format(cluster))
    exec('glove{}.save("glove{}.model")'.format(cluster, cluster))

Performing 200 training epochs with 3 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
Epoch 81
Epoch 82
Epoch 83
Epoch 84
Epoch 85
Epoch 86
Epoch 87
Epoch 88
Epoch 89
Epoch 90
Epoch 91
Epoch 92
Epoch 93
Epoch 94
Epoch 95
Epoch 96
Epoch 97
Epoch 98
Epoch 99
Epoch 100
Epoch 101
Epoch 102
Epoch 103
Epoch 104
Epoch 105
Epoc

# **Classification**

In [101]:
#To Import all the Clusters created using LDA based Topic Modeling
for c in range(10):
    exec('topic_{} = pd.read_csv("topic_{}.csv")'.format(c,c))
    exec("topic_{}= topic_{}.drop(columns=['Unnamed: 0'])".format(c,c))
    exec("topic_{}['Description'] = topic_{}['Description'].map(preprocess)".format(c,c))

In [102]:
#To Import all the trained GloVe models
for mod in range(10):  
    exec('glove{} = Glove.load("glove{}.model")'.format(mod, mod))

In [103]:
#This will return the index of cluster in which the master report of duplicate report may reside
def sim_with_clusters_lda_topn(DR, n):
    vec_bow = dictionary.doc2bow(DR)
    x= lda_model[vec_bow]
    topic = np.asarray(x)
    sim=[]
    x= topic[np.argsort(topic[:,1])[-n:][::-1],0]

    for i in range(len(x)):
        sim.append(int(x[i]))

    return sim

#To get Similarity between two feature vectors using the average of Cosine Similarity & Euclidean Similarity
def sim(vec1, vec2): 
    sim1 = 1/(1+np.linalg.norm(np.array(vec1) - np.array(vec2)))
    sim2 = cosine_similarity(vec1, vec2)
    sim=(sim1+sim2)/2 
    return sim

# **LDA and GLOVE**

In [104]:
#Returns Top-N Master Reports
def compare_topn(model, cluster, sent, DR, topn, modal):
    similarity=[]
    
    if (modal == 'single'):
        vec_duplicate, master= feature_vectors_single_modality(DR, sent, model)
    else:
        raise ValueError('Invalid Modality entered')

    for doc in range(len(master)):
        vec_master = master[doc]
        vec_master= [vec_master]
        unified_sim = sim(vec_duplicate, vec_master)
        similarity.append(unified_sim)
    
    similarity = np.asarray(similarity)
    similarity= np.concatenate(similarity, axis=0 )
    similarity= np.concatenate(similarity, axis=0 )
    max_similar_reports=similarity.argsort()[-topn:][::-1]

    return(max_similar_reports)

In [105]:
# creation of feature vectors by singlemodality feature extraction
def feature_vectors_single_modality(DR, corpus, model):
    master = averaged_word_vectorizer_glove(corpus=sent, model=model, num_features=100)

    vec_duplicate = averaged_word_vectorizer_glove(corpus=DR, model=model, num_features=100)

    vec_duplicate = [vec_duplicate]

    return vec_duplicate, master

def averaged_word_vectorizer_glove(corpus, model, num_features):
    vocabulary = set(model.dictionary)
    if(any(isinstance(i, list) for i in corpus)):
        features = [average_word_vectors_glove(tokenized_sentence, model, vocabulary, num_features)
                      for tokenized_sentence in corpus]
        return np.array(features)
    else:
          features = average_word_vectors_glove(corpus, model, vocabulary, num_features)
    return np.array(features)

def average_word_vectors_glove(words, model, vocabulary, num_features):  
    feature_vector = np.zeros((num_features,),dtype="float64")
    nwords = 0.  

    for word in words:
        if word in vocabulary: 
            nwords = nwords + 1.
            feature_vector = np.add(feature_vector, model.word_vectors[model.dictionary[word]])

    if nwords:
        feature_vector = np.divide(feature_vector, nwords)
        
    return feature_vector

# Test data

Determining the top-n values for the recall rate @ k

In [106]:
#Import Duplicate Reports
test = pd.read_csv('mobile_duplicate_reports.csv')
test = test.drop(columns=['Unnamed: 0'])
test['Description']= test['Description'].fillna('').astype(str).map(preprocess)
test = test.rename(columns={'Bug ID':'Bug_ID'})

#To Decide the Iterations
number_of_samples = test.shape[0]

if(number_of_samples > 200):
  number_of_samples = 200

In [107]:
#Evaluating the performance by Recall Rate
def evaluate(vec_acc, t2, t1):
  sum = 0
  for i,num in enumerate(vec_acc):
      sum = sum + int(num)
  recall_rate = (sum/len(vec_acc))*100
  print("Recall Rate : {} %".format(recall_rate))
  print("Time : ", (t2-t1)/60, "min")

In [108]:
#For k=1
vec_acc_top1=[]
t1_top1 = time.time()

for i in range(number_of_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test.Description[i] 
    n = 1
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]

    for max in max_cluster:
        exec('cluster = topic_{}'.format(max)) 
        exec('model = glove{}'.format(max))                
        exec('sent = topic_{}.Description'.format(max))
        
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})
        topn = 1  
        modal = 'single'                                 
        
        #This will return the Top-N predicted master reports
        max_sim = compare_topn(model, cluster, sent, sample, topn, modal)
        t2_top1 = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc_top1.append("0")
    else:
        vec_acc_top1.append("1")

Running sample 199

In [109]:
#For k=5
vec_acc_top5=[]
t1_top5 = time.time()

for i in range(number_of_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test.Description[i] 
    n = 3
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]

    for max in max_cluster:
        exec('cluster = topic_{}'.format(max)) 
        exec('model = glove{}'.format(max))                
        exec('sent = topic_{}.Description'.format(max))
        
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})
        topn = 1 
        modal = 'single'                                 
        
        #This will return the Top-N predicted master reports
        max_sim = compare_topn(model, cluster, sent, sample, topn, modal)
        t2_top5 = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc_top5.append("0")
    else:
        vec_acc_top5.append("1")

Running sample 199

In [110]:
#For k=10
vec_acc_top10=[]
t1_top10 = time.time()

for i in range(number_of_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test.Description[i] 
    n = 3
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]

    for max in max_cluster:
        exec('cluster = topic_{}'.format(max)) 
        exec('model = glove{}'.format(max))                
        exec('sent = topic_{}.Description'.format(max))
        
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})
        topn = 3  
        modal = 'single'                                 
        
        #This will return the Top-N predicted master reports
        max_sim = compare_topn(model, cluster, sent, sample, topn, modal)
        t2_top10 = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc_top10.append("0")
    else:
        vec_acc_top10.append("1")

Running sample 199

In [111]:
#For k=100
vec_acc_top100=[]
t1_top100 = time.time()

for i in range(number_of_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test.Description[i] 
    n = 3
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]

    for max in max_cluster:
        exec('cluster = topic_{}'.format(max)) 
        exec('model = glove{}'.format(max))                
        exec('sent = topic_{}.Description'.format(max))
        
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})
        topn = 33   
        modal = 'single'                                 
        
        #This will return the Top-N predicted master reports
        max_sim = compare_topn(model, cluster, sent, sample, topn, modal)
        t2_top100 = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc_top100.append("0")
    else:
        vec_acc_top100.append("1")

Running sample 199

In [112]:
print('For k=1')
evaluate(vec_acc_top1, t2_top1, t1_top1)

print('\nFor k=5')
evaluate(vec_acc_top5, t2_top5, t1_top5)

print('\nFor k=10')
evaluate(vec_acc_top10, t2_top10, t1_top10)

print('\nFor k=100')
evaluate(vec_acc_top100, t2_top100, t1_top100)

For k=1
Recall Rate : 0.0 %
Time :  0.9672367294629415 min

For k=5
Recall Rate : 2.5 %
Time :  2.6086865504582724 min

For k=10
Recall Rate : 10.0 %
Time :  2.554018723964691 min

For k=100
Recall Rate : 18.0 %
Time :  2.491474755605062 min


# **Textually Similar and Dissimilar Data**

# Textually Similar

In [113]:
#Import Textually Similar Data
test_sim = pd.read_csv('/content/drive/Shareddrives/DSCI-644-Team-5/PreProcessedData/Mobile_final_sim.csv')

#Cleaning the Data
test_sim = test_sim.drop(columns=['Unnamed: 0'])
test_sim['Description']= test_sim['Description'].fillna('').astype(str).map(preprocess)
test_sim.replace("", np.nan, inplace=True)
test_sim.dropna(subset = ["Duplicate_Bug_Ids"], inplace=True)
test_sim = test_sim.reset_index(drop=True)

#To Decide the Iterations
number_of_samples_sim = test_sim.shape[0]

if(number_of_samples_sim > 200):
  number_of_samples_sim = 200

In [114]:
#For k=1
vec_acc_top1_sim=[]
t1_top1_sim = time.time()

for i in range(number_of_samples_sim):
    print(f'\rRunning sample {i}', end='')
    sample = test_sim.Description[i] 
    n = 1
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]

    for max in max_cluster:
        exec('cluster = topic_{}'.format(max)) 
        exec('model = glove{}'.format(max))                
        exec('sent = topic_{}.Description'.format(max))
      
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})
        topn = 1   
        modal = 'single'                                 
        
        #This will return the Top-N predicted master reports
        max_sim = compare_topn(model, cluster, sent, sample, topn, modal)
        t2_top1_sim = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test_sim.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc_top1_sim.append("0")
    else:
        vec_acc_top1_sim.append("1")

Running sample 121

In [115]:
#For k=5
vec_acc_top5_sim=[]
t1_top5_sim = time.time()

for i in range(number_of_samples_sim):
    print(f'\rRunning sample {i}', end='')
    sample = test_sim.Description[i]
    n = 3
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]

    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))             
        exec('model = glove{}'.format(max))                
        exec('sent = topic_{}.Description'.format(max))     
        
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})
        topn = 1           
        modal = 'single'        
        
        #This will return the Top-N predicted master reports
        max_sim = compare_topn(model, cluster, sent, sample, topn, modal)
        t2_top5_sim = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test_sim.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc_top5_sim.append("0")
    else:
        vec_acc_top5_sim.append("1")

Running sample 121

In [116]:
#For k=10
vec_acc_top10_sim=[]
t1_top10_sim = time.time()

for i in range(number_of_samples_sim):
    print(f'\rRunning sample {i}', end='')
    sample = test_sim.Description[i] 
    n = 3
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]

    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))              
        exec('model = glove{}'.format(max))               
        exec('sent = topic_{}.Description'.format(max))    
        
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})
        topn = 3          
        modal = 'single'                                    
        
        #This will return the Top-N predicted master reports
        max_sim = compare_topn(model, cluster, sent, sample, topn, modal)
        t2_top10_sim = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test_sim.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc_top10_sim.append("0")
    else:
        vec_acc_top10_sim.append("1")

Running sample 121

In [117]:
#For k=100
vec_acc_top100_sim=[]
t1_top100_sim = time.time()

for i in range(number_of_samples_sim):
    print(f'\rRunning sample {i}', end='')
    sample = test_sim.Description[i]
    n = 3
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]

    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))              
        exec('model = glove{}'.format(max))               
        exec('sent = topic_{}.Description'.format(max))     
        
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})
        topn = 33          
        modal = 'single'    

        #This will return the Top-N predicted master reports
        max_sim = compare_topn(model, cluster, sent, sample, topn, modal)
        t2_top100_sim = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test_sim.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc_top100_sim.append("0")
    else:
        vec_acc_top100_sim.append("1")

Running sample 121

In [118]:
print('For k=1')
evaluate(vec_acc_top1_sim, t2_top1_sim, t1_top1_sim)

print('\nFor k=5')
evaluate(vec_acc_top5_sim, t2_top5_sim, t1_top5_sim)

print('\nFor k=10')
evaluate(vec_acc_top10_sim, t2_top10_sim, t1_top10_sim)

print('\nFor k=100')
evaluate(vec_acc_top100_sim, t2_top100_sim, t1_top100_sim)

For k=1
Recall Rate : 0.0 %
Time :  0.5513003865877787 min

For k=5
Recall Rate : 2.459016393442623 %
Time :  1.0479019204775493 min

For k=10
Recall Rate : 13.114754098360656 %
Time :  0.9820038596789042 min

For k=100
Recall Rate : 22.950819672131146 %
Time :  1.163325806458791 min


# Textually Dissimilar

In [119]:
#Import Textually Similar Data
test_dissim = pd.read_csv('/content/drive/Shareddrives/DSCI-644-Team-5/PreProcessedData/Mobile_final_dis.csv')

#Cleaning the Data
test_dissim = test_dissim.drop(columns=['Unnamed: 0'])
test_dissim['Description']= test_dissim['Description'].fillna('').astype(str).map(preprocess)
test_dissim.replace("", np.nan, inplace=True)
test_dissim.dropna(subset = ["Duplicate_Bug_Ids"], inplace=True)
test_dissim = test_dissim.reset_index(drop=True)

#To Decide the Iterations
number_of_samples_dis = test_dissim.shape[0]

if(number_of_samples_dis > 200):
  number_of_samples_dis = 200

In [120]:
#For k=1
vec_acc_top1_dis=[]
t1_top1_dis = time.time()

for i in range(number_of_samples_dis):
    print(f'\rRunning sample {i}', end='')
    sample = test_dissim.Description[i] 
    n = 1
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]
    
    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))              
        exec('model = glove{}'.format(max))               
        exec('sent = topic_{}.Description'.format(max))     
        
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})
        topn = 1              
        modal = 'single'                                    
        
        #This will return the Top-N predicted master reports
        max_sim = compare_topn(model, cluster, sent, sample, topn, modal)
        t2_top1_dis = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test_dissim.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc_top1_dis.append("0")
    else:
        vec_acc_top1_dis.append("1")

Running sample 130

In [121]:
#For k=5
vec_acc_top5_dis=[]
t1_top5_dis = time.time()

for i in range(number_of_samples_dis):
    print(f'\rRunning sample {i}', end='')
    sample = test_dissim.Description[i] 
    n = 3
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]

    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))                       
        exec('model = glove{}'.format(max))                
        exec('sent = topic_{}.Description'.format(max))     
        
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})
        topn = 1          
        modal = 'single'                                    
        
        #This will return the Top-N predicted master reports
        max_sim = compare_topn(model, cluster, sent, sample, topn, modal)
        t2_top5_dis = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test_dissim.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc_top5_dis.append("0")
    else:
        vec_acc_top5_dis.append("1")

Running sample 130

In [122]:
#For k=10
vec_acc_top10_dis=[]
t1_top10_dis = time.time()

for i in range(number_of_samples_dis):
    print(f'\rRunning sample {i}', end='')
    sample = test_dissim.Description[i] 
    n = 3
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]

    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))                         
        exec('model = glove{}'.format(max))                
        exec('sent = topic_{}.Description'.format(max))     
        
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})
        topn = 3      
        modal = 'single'                                  
        
        #This will return the Top-N predicted master reports
        max_sim = compare_topn(model, cluster, sent, sample, topn, modal)
        t2_top10_dis = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test_dissim.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc_top10_dis.append("0")
    else:
        vec_acc_top10_dis.append("1")

Running sample 130

In [123]:
#For k=100
vec_acc_top100_dis=[]
t1_top100_dis = time.time()

for i in range(number_of_samples_dis):
    print(f'\rRunning sample {i}', end='')
    sample = test_dissim.Description[i] 
    n = 3
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]

    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))                
        exec('model = glove{}'.format(max))               
        exec('sent = topic_{}.Description'.format(max))    
        
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})
        topn = 33         
        modal = 'single'                                   
        
        #This will return the Top-N predicted master reports
        max_sim = compare_topn(model, cluster, sent, sample, topn, modal)
        t2_top100_dis = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test_dissim.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc_top100_dis.append("0")
    else:
        vec_acc_top100_dis.append("1")

Running sample 130

In [124]:
print('For k=1')
evaluate(vec_acc_top1_dis, t2_top1_dis, t1_top1_dis)

print('\nFor k=5')
evaluate(vec_acc_top5_dis, t2_top5_dis, t1_top5_dis)

print('\nFor k=10')
evaluate(vec_acc_top10_dis, t2_top10_dis, t1_top10_dis)

print('\nFor k=100')
evaluate(vec_acc_top100_dis, t2_top100_dis, t1_top100_dis)

For k=1
Recall Rate : 0.0 %
Time :  0.6100683053334554 min

For k=5
Recall Rate : 4.580152671755725 %
Time :  1.7570760011672975 min

For k=10
Recall Rate : 6.106870229007633 %
Time :  1.7614442745844523 min

For k=100
Recall Rate : 13.740458015267176 %
Time :  1.8518136620521546 min
